In [1]:
!pip install PyGSP
!pip install tensorboardx
!pip install torch
!pip install neuralforecast
!pip install pytorch-tcn

In [2]:
!pip install -U giotto-tda

In [3]:
import torch
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
import networkx as nx
from gtda.graphs import KNeighborsGraph
import itertools
from pytorch_tcn import TCN


#import os
#import sys
#sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

from importlib import reload
import Model2
reload(Model2)
import Model3
reload(Model3)

from utils import GraphRNN_dataset, GraphRNN_DataSampler
from Model2 import STGCNChebGraphConv
from Model3 import ParametricNetWithPooling
import matplotlib.pyplot as plt

In [39]:
print("Starting training run...")
flow_dataset = "../data/daily_county2county_2019_01_01.csv"
epi_dataset = "../data_epi/epidemiology.csv"
epi_dates = ["2020-06-09", "2020-06-10", "2020-06-11", "2020-06-12",
             "2020-06-13", "2020-06-14", "2020-06-15", "2020-06-16",
             "2020-06-17", "2020-06-18", "2020-06-19", "2020-06-20",
             "2020-06-21", "2020-06-22", "2020-06-23", "2020-06-24",
                "2020-06-25", "2020-06-26", "2020-06-27", "2020-06-28",
                "2020-06-29", "2020-06-30", "2020-07-01", "2020-07-02",
                "2020-07-03", "2020-07-04", "2020-07-05", "2020-07-06",
                "2020-07-07", "2020-07-08", "2020-07-09", "2020-07-10",
                "2020-07-11", "2020-07-12", "2020-07-13", "2020-07-14",
                "2020-07-15", "2020-07-16", "2020-07-17", "2020-07-18",
                "2020-07-19", "2020-07-20", "2020-07-21", "2020-07-22",
                "2020-07-23", "2020-07-24", "2020-07-25", "2020-07-26",
                "2020-07-27", "2020-07-28", "2020-07-29", "2020-07-30"
             ]


input_hor = 5
pred_hor = 1
print("Loading data...")
data_set = GraphRNN_dataset(epi_dates = epi_dates,
                            flow_dataset = flow_dataset,
                            epi_dataset = epi_dataset,
                            input_hor=input_hor,
                            pred_hor=pred_hor,
                            fake_data=False)
# data_set.visualize(0)
device = 'cpu'#torch.device("cuda" if torch.cuda.is_available() else "cpu")


# data_sampler = GraphRNN_DataSampler(data_set, input_hor=input_hor, pred_hor=pred_hor)
data_loader = torch.utils.data.DataLoader(data_set, batch_size=10, pin_memory=True, num_workers=0, shuffle=True)
for thing in data_loader:
    for batch in thing:
        batch.to(device)


print("Data loaded.")
input_edge_weights, input_node_data, target_edge_weights, target_node_data = next(iter(data_loader))



Starting training run...
Loading data...
Flow data shape:      geoid_o  geoid_d      lng_o      lat_o      lng_d      lat_d        date  \
9       1001     1021 -86.642757  32.534921 -86.718814  32.847853  2019-01-01   
21      1001     1051 -86.642757  32.534921 -86.149147  32.596648  2019-01-01   
38      1001     1101 -86.642757  32.534921 -86.207614  32.220258  2019-01-01   
265     1003     1035 -87.749845  30.660974 -86.993681  31.429267  2019-01-01   
274     1003     1053 -87.749845  30.660974 -87.161620  31.126123  2019-01-01   

     visitor_flows  pop_flows  
9         0.463154   0.550704  
21        0.688665   0.725945  
38        0.647880   0.694244  
265       0.379613   0.474777  
274       0.495607   0.564922  
Signals[0] data shape:          geoid_o        date  new_confirmed  infection_gone
9897156     2013  2020-06-09            NaN             0.0
9897865     2016  2020-06-09            0.0             0.0
9898658     2020  2020-06-09            5.0             1.0


100%|███████████████████████████████████████████| 14328/14328 [00:03<00:00, 4373.99it/s]


node_data: torch.Size([52, 3070, 1])
edge_weights: torch.Size([52, 14328, 3])
Number of unique nodes from final edges: 3070
Number of unique nodes from signals: 3070
Node IDs match between edges and signals
Sparsity of node data:  0.36212728639438735
Sparsity of edge weights:  0.0
Data loaded.


In [5]:

print(input_edge_weights[0, 0, 0:100])
for batch in data_loader:
    print(len(batch))
    print(batch[0].shape)
    print(batch[-1].shape)
    break
for batch in input_node_data:
    print(batch.shape)
    for input_type in batch:
        #print(input_type.shape)
        #print(input_type)
        break
    break

tensor([[1.0010e+03, 1.0210e+03, 5.5070e-01],
        [1.0010e+03, 1.0510e+03, 7.2594e-01],
        [1.0010e+03, 1.1010e+03, 6.9424e-01],
        [1.0030e+03, 1.0350e+03, 4.7478e-01],
        [1.0030e+03, 1.0530e+03, 5.6492e-01],
        [1.0030e+03, 1.0970e+03, 7.1825e-01],
        [1.0030e+03, 1.1290e+03, 4.3224e-01],
        [1.0030e+03, 1.2033e+04, 7.0069e-01],
        [1.0030e+03, 1.2113e+04, 5.2038e-01],
        [1.0030e+03, 2.8047e+04, 5.7406e-01],
        [1.0030e+03, 1.0990e+03, 4.7646e-01],
        [1.0070e+03, 1.0730e+03, 6.2993e-01],
        [1.0070e+03, 1.1050e+03, 4.6406e-01],
        [1.0070e+03, 1.1170e+03, 5.9333e-01],
        [1.0070e+03, 1.1250e+03, 6.4533e-01],
        [1.0090e+03, 1.0430e+03, 6.4440e-01],
        [1.0090e+03, 1.0730e+03, 7.5343e-01],
        [1.0090e+03, 1.0950e+03, 6.1876e-01],
        [1.0110e+03, 1.0870e+03, 4.4859e-01],
        [1.0110e+03, 1.1010e+03, 5.3985e-01],
        [1.0110e+03, 1.1090e+03, 4.6189e-01],
        [1.0130e+03, 1.0350e+03, 4

In [33]:
def train(model, data_loader, criterion, optimizer, pred_hor, device, n_epochs=10, save=None):
    losses = []
    parameter_mag = {param_name: [] for param_name, param in model.named_parameters()}
    for epoch in range(n_epochs):
        epoch_loss = 0
        batch_num = 0
        for i, (input_edge_weights, input_node_data, target_edge_weights, target_node_data) in enumerate(tqdm(data_loader)):
            if batch_num == 1 and epoch == 1:
                prof.step()
            for param_name, param in model.named_parameters():
                parameter_mag[param_name].append(param.abs().mean().item())
            
            # input_edge_weights = input_edge_weights[i]
            # input_node_data = input_node_data[i]
            # target_edge_weights = target_edge_weights[i]
            # target_node_data = target_node_data[i]
            
            input_edge_weights = input_edge_weights.to(device)
            input_node_data = input_node_data.to(device)
            target_edge_weights = target_edge_weights.to(device)
            target_node_data = target_node_data.to(device)
            # output = model(x_in=input_node_data, edge_weights = input_edge_weights, pred_hor = pred_hor)
            #print(input_edge_weights.shape)
            #print(input_node_data.shape)
            #print(target_edge_weights.shape)
            #print(target_node_data.shape)
            output = model(x_in=input_node_data)

            
            # print(f"output: {output}")
            # print(f"target_node_data: {target_node_data}")
              
            loss = criterion(output[-pred_hor:,:,:], target_node_data[:pred_hor,:,:])
            loss += 0 * criterion(output[:input_hor,:,:], input_node_data[:,:])
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            batch_num += 1
        epoch_loss = epoch_loss/((pred_hor) * target_node_data.shape[2])
        losses.append(epoch_loss)
        print(f"EPOCH: {epoch} ", end="")
        
        print(f"$ Loss: { epoch_loss:.3e} ")
        
        print(f"Input: {input_node_data[:, :5, 0].int()} ")
        print(f"$ Output: {output[-pred_hor:, :5, 0].int()} ")
        print(f"$ Target: {target_node_data[:, :5, 0].int()}")
    if save is not None:
        torch.save(model.state_dict(), save)
    return losses, parameter_mag

In [42]:
#reimport models
import Model2
reload(Model2)
import Model3
reload(Model3)
import GCNN_RNN
reload(GCNN_RNN)
from utils import GraphRNN_dataset, GraphRNN_DataSampler
from Model2 import STGCNChebGraphConv
from Model3 import ParametricNetWithPooling
from GCNN_RNN import GCNN_RNN

models  = [
            # TCN(
            #     num_inputs = data_set.n_features,
            #     num_channels = [3070, 1, 4],
            #     kernel_size = 1, #kernel_size: int = 4,
            #     input_shape = 'NLC', #input_shape: str = 'NCL',
            #     lookahead = 7, # default: 0
            #     #dilations: Optional[ ArrayLike ] = None,
            #     #dilation_reset: Optional[ int ] = None,
            #     #dropout: float = 0.1,
            #     #causal: bool = True,
            #     #use_norm: str = 'weight_norm',
            #     #activation: str = 'relu',
            #     #kernel_initializer: str = 'xavier_uniform',
            #     #use_skip_connections: bool = False,
            #     #embedding_shapes: Optional[ ArrayLike ] = None,
            #     #embedding_mode: str = 'add',
            #     #use_gate: bool = False,
            #     #output_projection: Optional[ int ] = None,
            #     #output_activation: Optional[ str ] = None,
            # ),
            GCNN_RNN(n_nodes = data_set.n_nodes,
                    n_features = data_set.n_features,
                    n_out_features = 50,
                    h_size = 50,
                    input_hor = 7,
                    device=device,
                    dtype=torch.float32,
                    fixed_edge_weights = input_edge_weights[0,0,:,:]),
           # ParametricNetWithPooling(n_nodes = data_set.n_nodes,
           #         n_features = data_set.n_features,
           #         h_size = 50,
           #         f_out_size = 50,
           #         fixed_edge_weights = input_edge_weights[0,0,:,:],
           #         device=device,
           #         dtype=torch.float32)
]
model_output = []

# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
for i, model in enumerate(models):
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    criterion = torch.nn.MSELoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.00045)


    torch.autograd.set_detect_anomaly(False)

    model.to(device)
    
    log_dir = './log'
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    # Use torch.profiler to profile the model
    with torch.profiler.profile(
        activities=[
            torch.profiler.ProfilerActivity.CPU,
            torch.profiler.ProfilerActivity.CUDA,
        ],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=2, repeat=0),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log'),
        record_shapes=True,
        profile_memory=False,
        with_stack=True
    ) as prof:
        print("Starting training with profiling...")
        losses, parameter_mag = train(model, data_loader,
                                  criterion, optimizer,
                                  pred_hor, device, n_epochs=250,
                                  save="model_state_dict.pth")
        model_output.append((losses, parameter_mag))
        print("Finished training with profiling.")


    # Verify that the log directory is populated
    if os.listdir(log_dir):
        print(f"Log files generated in {log_dir}")
    else:
        print(f"No log files found in {log_dir}")
    

    print("Storing model state...")
    torch.save(model.state_dict(), f"model_{i}_state_dict.pth")
    print("Model state stored.")

Starting training with profiling...


  0%|                                                             | 0/5 [00:24<?, ?it/s]

torch.Size([50, 3070, 50])
tensor([[[ 9.6505e+00,  3.6215e+00,  1.1069e+00,  ...,  2.8452e+00,
           4.0214e+00, -9.3459e+00],
         [ 5.0665e+01,  1.9013e+01,  5.8112e+00,  ...,  1.4937e+01,
           2.1112e+01, -4.9065e+01],
         [ 1.6309e+00,  6.1202e-01,  1.8706e-01,  ...,  4.8082e-01,
           6.7961e-01, -1.5794e+00],
         ...,
         [ 8.4184e-01,  3.1591e-01,  9.6557e-02,  ...,  2.4819e-01,
           3.5080e-01, -8.1526e-01],
         [ 8.8288e-01,  3.3132e-01,  1.0127e-01,  ...,  2.6029e-01,
           3.6790e-01, -8.5501e-01],
         [ 9.1080e-01,  3.4179e-01,  1.0447e-01,  ...,  2.6852e-01,
           3.7954e-01, -8.8205e-01]],

        [[ 7.1470e+00,  2.6820e+00,  8.1975e-01,  ...,  2.1071e+00,
           2.9782e+00, -6.9213e+00],
         [ 3.0681e+01,  1.1514e+01,  3.5191e+00,  ...,  9.0454e+00,
           1.2785e+01, -2.9713e+01],
         [ 2.9663e+00,  1.1131e+00,  3.4023e-01,  ...,  8.7452e-01,
           1.2361e+00, -2.8726e+00],
         ...

RuntimeError: shape '[10, 7, 3070, 50]' is invalid for input of size 7675000

In [ ]:
for losses, parameter_mag in model_output:    
    print("Plotting losses...")
    plt.plot(losses)
    plt.xlabel("Iteration")
    plt.ylabel("Loss")
    plt.yscale("log")
    plt.show()
    
    print("Plotting parameter magnitudes...")
    plt.figure()
    for key in parameter_mag:
        plt.plot(parameter_mag[key], label=key)
    
    plt.xlabel("Iteration")
    plt.ylabel("Parameter Magnitude")
    # plt.yscale("log")
    plt.legend()
    plt.show()